# Kaggle - Santander Product Recommendation Baseline Modeling
**Author: Chris Shin**

# Data Preprocessing

Items to do during preprocessing:
- impute missing values
    - for products, we can replace missing values with 0 indicating the products were not purchased
- preprocess categorical and numerical data.
    - Categorical data: performs Label Encoding through factorize(). 
    - Numerical data: converted to integer data for those data type is expressed as an object. 
- combine train and test data in order to provide same data cleaning process. 

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2023)

trn = pd.read_csv('./data/train_ver2.csv')
tst = pd.read_csv('./data/test_ver2.csv')

# separates product features
prods = trn.columns[24:].tolist()

# replace missing values of 0 -> indicating no purchase
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# if the product is all 0, that means all customers did not purchase. We can remove those products
no_product = (trn[prods].sum(axis=1) == 0)
trn = trn[~no_product]

# data that are not in test data are marked as 0.
# Then combine train and test data
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# features to use for modeling
features = []

# perform label encoding
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi',
                    'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

C:\Users\Yuchie\AppData\Local\Temp\ipykernel_12896\2022540250.py:7: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  trn = pd.read_csv('./data/train_ver2.csv')
C:\Users\Yuchie\AppData\Local\Temp\ipykernel_12896\2022540250.py:8: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  tst = pd.read_csv('./data/test_ver2.csv')
C:\Users\Yuchie\AppData\Local\Temp\ipykernel_12896\2022540250.py:33: FutureWarning: Specifying the specific value to use for `na_sentinel` is deprecated and will be removed in a future version of pandas. Specify `use_na_sentinel=True` to use the sentinel value -1, and `use_na_sentinel=False` to encode NaN values.
  df[col], _ = df[col].factorize(na_sentinel=-99)
C:\Users\Yuchie\AppData\Local\Temp\ipykernel_12896\2022540250.py:33: FutureWarning: Specifying the specific value to use for `na_sentinel` is deprecated and will be removed in a future version of panda

In [2]:
df['age'].unique()

array([' 35', ' 23', ' 22', ' 24', ' 65', ' 28', ' 25', ' 26', ' 53',
       ' 27', ' 32', ' 37', ' 31', ' 39', ' 63', ' 33', ' 55', ' 42',
       ' 58', ' 38', ' 50', ' 30', ' 45', ' 44', ' 36', ' 29', ' 60',
       ' 57', ' 67', ' 47', ' NA', ' 34', ' 48', ' 46', ' 54', ' 84',
       ' 15', ' 12', '  8', '  6', ' 83', ' 40', ' 77', ' 69', ' 52',
       ' 59', ' 43', ' 10', '  9', ' 49', ' 41', ' 51', ' 78', ' 16',
       ' 11', ' 73', ' 62', ' 66', ' 17', ' 68', ' 82', ' 95', ' 96',
       ' 56', ' 61', ' 79', ' 14', ' 19', ' 13', ' 86', ' 64', ' 20',
       ' 72', ' 89', ' 71', '  7', ' 70', ' 74', ' 21', ' 18', ' 75',
       '  4', ' 80', ' 81', '  5', ' 76', ' 92', ' 93', ' 85', ' 91',
       ' 87', ' 90', ' 94', ' 99', ' 98', ' 88', ' 97', '100', '101',
       '106', '103', '  3', '  2', '102', '104', '111', '107', '109',
       '105', '112', '115', '110', '116', '108', '113', 37, 81, 43, 30,
       41, 67, 59, 46, 36, 47, 69, 39, 44, 38, 34, 42, 31, 40, 48, 54, 51,
       33, 62

In [3]:
df['antiguedad'].unique()

array(['      6', '     35', '     34', '     NA', '     33', '     31',
       '     21', '     16', '     27', '      9', '     22', '     13',
       '     29', '      8', '     11', '     10', '     28', '     24',
       '      7', '     25', '     14', '     12', '     26', '     23',
       '      1', '     18', '      4', '      3', '     17', '     32',
       '     20', '     15', '     30', '     19', '    157', '     36',
       '     40', '     38', '     37', '     39', '      0', '      5',
       '     47', '     44', '     42', '     46', '     45', '     43',
       '     41', '     57', '     48', '     52', '     49', '     50',
       '     56', '     58', '     51', '     55', '     54', '     53',
       '     59', '     62', '     61', '     60', '     63', '      2',
       '    139', '    165', '    118', '    164', '     94', '    159',
       '    143', '    105', '    151', '    162', '    137', '    150',
       '    128', '    122', '    156', '    119', 

In [4]:
df['renta'].unique()

array([87218.1, 35548.74, 122179.11000000002, ..., '  139164.12',
       '  100647.45', '   72765.27'], dtype=object)

In [5]:
df['indrel_1mes'].unique()

array([1.0, nan, 3.0, 2.0, '1.0', '1', '3', '3.0', '2.0', 'P', '4', 4.0,
       '4.0', '2'], dtype=object)

### Missing Values

In [6]:
# Replace singular and missing values ​​of numeric variables with -99 and convert them to integers.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# Numerical variables to be used for learning are sought in features.
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

# Feature Engineering

For the baseline model we will use 24 customer features and 4 derivated features from date. Lastely we will use lag value of those 24 customer features to show the previous information.

Year and month information are extracted from the variables fecha_alta, which means the date the customer signed the first contract, and ult_fec_cli_1t, which means the date the customer last received the first grade. There are other possible features can be generated reliated with date, such as season or special date like vacation. These can be added after the baseline model

In time series data, various derived variables can be created based on the customer's past data. For example, whether the customer's age has changed in the last 3 months (wheter there is birthday or not) can be created as a binary variable, information on products purchased a month ago can be used as a variable, or the average monthly salary for the last 6 months can be calculated.

For baseline, we only have -1 lag value but we can also use 2 or 3 lag values to see how customer purchased new products from 2 or 3 months before. For baseline model we only use 1 lag value, but we can add more depends on the performance of the baseline model

In [7]:
# Extract year and month information from two date variables.
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

# All other missing values ​​are replaced with -99.
df.fillna(-99, inplace=True)

def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# Create a lag by copying the data and adding 1 to the int_date date. Add _prev to the variable name.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

# Merge original data and lag data based on ncodeper and int_date. Since the int_date of the lag data is pushed back by 1, the last month's product information is inserted.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

# delete after merge for the memory optimization
del df, df_lag

# Replace with 0 in case product information for the previous month does not exist.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

# Add the lag-1 variable.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]


After the baseline model, we should check the results and think of additional featuere engineering and update accordingly. 

# Model Training

In this prject, a total of 1 year and 6 months of data from 2015-01-28 to 2016-05-28 is provided as training data, and the test data to be predicted is future data from 2016-06-28. Therefore, in the verification process, 2016-05-28, the most recent data in the training data, is separated as verification data, and the rest is used for training.

For simplicity, we use data from 2016-01-28 to 2016-05-28 for baseline model training. We will add all dates after we modifying the feature engineering and the model hyperparamter turnings.

In [8]:

# For training, we use date from 2016-01-28 to 2016-05-28 month data.
# For validation, we use 2016-05-28 as a validation
# Lastly, for test data, we will use 2016-06-28.
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

# Extract only the number of new purchases from the training data.
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]



### Model

For our baseline model we will use one of the popular machine learning algorithm XGBoost that is widely used in Kaggle as well. It is also great for constructing baseline model as it is fast. Modern libraries like XGBoost come equipped with several speed enhancements, making it possible to train a well-performing model in a short amount of time. XGBoost and similar libraries can also be trained on GPUs (some assembly required), making training even faster on larger datasets.

The XGBoost also good for time series forecasting model as it is able to produce reasonable forecasts right out of the box with no hyperparameter tuning. One of the key advantages of XGBoost is its ability to handle missing data and large datasets efficiently. It also has a number of hyperparameters that can be tuned to improve model performance, including the learning rate, depth of the trees, and regularization parameters

The caveat is that most people tend to spend too much time finding the optimal parameters for their models. This task is called hyperparameter tuning. Finding good parameters can yield significant performance improvements. However, in terms of efficiency versus time investment, it is important to spend more time on feature engineering rather than hyperparameter tuning.

In [9]:

# Construct the parameter for the XGB Model
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

# Convert training and validation data to XGBoost format.
X_trn = XY_trn[features].values
Y_trn = XY_trn['y'].values
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld[features].values
Y_vld = XY_vld['y'].values
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

# Save model in the pickle for easy loading for the future
import pickle
pickle.dump(model, open("./model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit


[18:58:39] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-mlogloss:2.70583	eval-mlogloss:2.76067
[1]	train-mlogloss:2.44127	eval-mlogloss:2.49695
[2]	train-mlogloss:2.26188	eval-mlogloss:2.31725
[3]	train-mlogloss:2.12551	eval-mlogloss:2.18293
[4]	train-mlogloss:2.01400	eval-mlogloss:2.07096
[5]	train-mlogloss:1.91945	eval-mlogloss:1.97631
[6]	train-mlogloss:1.84273	eval-mlogloss:1.89985
[7]	train-mlogloss:1.77555	eval-mlogloss:1.83276
[8]	train-mlogloss:1.71464	eval-mlogloss:1.77159
[9]	train-mlogloss:1.66296	eval-mlogloss:1.71996
[10]	train-mlogloss:1.61705	eval-mlogloss:1.67376
[11]	train-mlogloss:1.57471	eval-mlogloss:1.63104
[12]	train-mlogloss:1.53632	eval-mlogloss:1.59223
[13]	train-mlogloss:1.50267	eval-mlogloss:1.55879
[14]	train-mlogloss:1.47280	eval-mlogloss:1.52845
[15]	train-mlogloss:1.44417	eval-mlogloss:1.49898
[16]	train-mlo

### Cross Validation

In cross-validation, the performance level is confirmed using MAP@7, the evaluation scale of this project.

In [10]:
import numpy as numpy

# average precision
def apk(actual, predicted, k=7, default=0.0):
    # Since it is MAP@7, we use max 7 items to calculate
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        # Rules to give a score:
        # Predicted is in the actual (p in actual)
        # predicted is not a duplicate (p no in predicted[:i])
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # if actual is empty, it return 0
    if not actual:
        return default

    # use number of actual to calculkate the average precision
    return score / min(len(actual) , k)

# mean average precision
def mapk(actual, predicted, k=7, default=0.0):
    # actual and predicted are list of list. 
    # for each customer, get the average precision and get mean using np.mean()
    return np.mean([apk(a, p, k, default) for a,p in zip(actual, predicted)])

In [11]:
# Code below is to selectin top 7 predictions for MAP@7 evalution metrics
# Extract customer identification number.
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld['ncodpers'].values
# Retrieve new purchases from validation data.
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]    
add_vld = vld[[prod + '_add' for prod in prods]].values
add_vld_list = [list() for i in range(len(ncodpers_vld))]

# The new purchase correct answer value for each customer is stored in add_vld_list,
# and the total count is stored in count_vld
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

# The highest score of MAP@7 that can be obtained from the verification data is obtained in advance
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

# Get the predicted value for the validation data.
X_vld = XY_vld[features].values
Y_vld = XY_vld['y'].values
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

# Since new purchases are not possible for products held last month, subtract 1 from the predicted value in advance.
preds_vld = preds_vld - XY_vld[[prod + '_prev' for prod in prods]].values

# Extract the top 7 prediction data from validation data
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])
    
# MAP@7 for validation
print(mapk(add_vld_list, result_vld, 7, 0.0))


C:\Users\Yuchie\AppData\Local\Temp\ipykernel_12896\1234760100.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vld[padd] = vld[prod] - vld[prev]
C:\Users\Yuchie\AppData\Local\Temp\ipykernel_12896\1234760100.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vld[padd] = vld[prod] - vld[prev]
C:\Users\Yuchie\AppData\Local\Temp\ipykernel_12896\1234760100.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


0.04266379915553903


c:\Users\Yuchie\Desktop\DataScience\santander\Lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


0.012745271121263942


In [12]:
vld[features]

,ind_empleado,pais_residencia,sexo,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,...,ind_hip_fin_ult1_prev,ind_plan_fin_ult1_prev,ind_pres_fin_ult1_prev,ind_reca_fin_ult1_prev,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev
10394531,0,0,0,0,0,0,-99,101,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10394532,0,0,1,0,0,0,-99,45,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10394533,0,0,1,0,0,0,-99,101,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
10394534,0,0,0,0,0,0,-99,5,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10394535,0,0,1,0,0,0,-99,5,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11091065,0,0,1,1,0,0,-99,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11091066,0,0,1,1,0,0,-99,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11091067,0,0,0,0,0,0,-99,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11091068,0,0,0,1,0,0,-99,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The MAP@7 rating scale fluctuates according to the data with the highest score. The highest score of MAP@7 that can be obtained from the validation data of this baseline model is 0.042633. Our baseline model is

# Test Prediction

In [ ]:
X_all = XY[features].values
Y_all = XY['y'].values
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]

# Increase Number of tress propertional to the increased size of data
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))

# retrain the xgb model
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

# Check the feature importance to see if feature that are assumed to be important listed on top
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

# calculate test data prediction result
X_tst = tst[features].values
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst['ncodpers'].values
preds_tst = preds_tst - tst[[prod + '_prev' for prod in prods]].values

# submission file creation
submit_file = open('./model/xgb.baseline.2023-03-24.csv', 'w')
submit_file.write('ncodpers,added_products\n')

numRows = 0
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))
    numRows += 1
print(numRows)